<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content/blob/master/tutorials/W1D2_ModelingPractice/W1D2_Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Framing the Question
**Week 1, Day 2: Modeling Practice**

**By Neuromatch Academy**

__Content creators:__ Marius 't Hart, Megan Peters, Paul Schrater, Gunnar Blohm

**Our 2021 Sponsors, including Presenting Sponsor Facebook Reality Labs**

<p align='center'><img src='https://github.com/NeuromatchAcademy/widgets/blob/master/sponsors.png?raw=True'/></p>

---
# Tutorial objectives
Yesterday you gained some understanding of what models can buy us in neuroscience. But how do you build a model? Today, we will try to clarify the process of computational modeling, by thinking through the logic of modeling based on your project ideas.

We assume that you have a general idea of a project in mind, i.e. a preliminary question, and/or phenomenon you would like to understand. You should have started developing a project idea yesterday with [this brainstorming demo](https://youtu.be/H6rSlZzlrgQ). Maybe you have a goal in mind. We will now work through the 4 first steps of modeling ([Blohm et al., 2019](https://doi.org/10.1523/ENEURO.0352-19.2019)): 

**Framing the question**

1. finding a phenomenon and a question to ask about it
2. understanding the state of the art
3. determining the basic ingredients
4. formulating specific, mathematically defined hypotheses

The remaining steps 5-10 will be covered in a second notebook that you can consult throughout the modeling process when you work on your projects.

**Importantly**, we will guide you through Steps 1-4 today. After you do more work on projects, you likely have to revite some or all of these steps *before* you should move on the the remaining steps of modeling. 

**Note**: there will be no coding today. It's important that you think through the different steps of this how-to-model tutorial to maximize your chance of succeeding in your group projects.

**Think! Sections**: All activities you should perform are labeled with **Think!**. These are discussion based exercises and can be found in the Table of Content on the left side of the notebook. Make sure you complete all within a section before moving on!

Enjoy!


----
#Step 1: Finding a phenomenon and a question to ask about it


In [ ]:
# @title Video 1: Asking a question
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
      def __init__(self, id, page=1, width=400, height=300, **kwargs):
          self.id=id
          src = 'https://player.bilibili.com/player.html?bvid={0}&page={1}'.format(id, page)
          super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id="", width=854, height=480, fs=1)
  print('Video available at https://www.bilibili.com/video/{0}'.format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id="Prf_Tc9UNp0", width=854, height=480, fs=1, rel=0)
  print('Video available at https://youtube.com/watch?v=' + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

## Think! 1: Asking your own question
You should already have a project idea from your brainstorming yesterday. **Write down the phenomenon, question and goal(s) if you have them.** 

As a reminder, here is what you should discuss and write down:

* What exact aspect of data needs to be modelled?
  * Answer this question clearly and precisely!
Otherwise you will get lost (almost guaranteed)
  * Write everything down!
  * Also identify aspects of data that you do not want to address (yet)
* Define the model evaluation method!
  * How will you know your model is good?
  * E.g. comparison to specific data (quantitative method of comparison?)
* Think of an experiment that could test your model
  * You essentially want your model to interface with this experiment, i.e. you want to simulate this experiment

You can find interesting questions by looking for phenomena that differ from your expectations. In *what* way does it differ? *How* could that be explained (starting to think about mechanistic questions and structural hypotheses)? *Why* could it be the way it is? What experiment could you design to investigate this phenomenon? What kind of data would you need?

>### Avoid these common pitfalls
* Question is too general
  * Remember: science advances one small step at the time. Get the small step right…
* Precise aspect of phenomenon you want to model is unclear
  * You fail to ask a meaningful question
* You have already chosen a toolkit
  * This will prevent you from thinking deeply about the best way to answer your scientific question
* You don’t have a clear goal
  * What do you want to get out the model?
* You don’t have a potential experiment in mind
  * This will help concretize your objectives and think through the logic behind your goal

**This should take no more than 30 minutes.**

----
#Step 2: Understanding the state of the art


Here you will do a literature review (**to be done AFTER this tutorial!**). For the projects, do not spend too much time on this. A thorough literature review should take weeks or months depending on your prior knowledge of the field...

The important thing for your project here is not to exhaustively survey the literatire but rather to learn the process of modeling. 1-2 days of digging into the literature should be enough!

**Here is what you should get out of it**:
* Survey the literature
  * What’s known?
  * What has already been done?
  * Previous models as a starting point?
  * What hypotheses have been emitted in the field?
  * Are there any alternative / complementary models?
* What skill sets are required?
  * Do I need learn something before I can start?
  * Ensures that no important aspect is missed
* Provides specific data sets / alternative models for comparison

**Do this AFTER the tutorial**


----
#Step 3: Determining the basic ingredients


In [ ]:
# @title Video 2: Determining basic ingredients
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
      def __init__(self, id, page=1, width=400, height=300, **kwargs):
          self.id=id
          src = 'https://player.bilibili.com/player.html?bvid={0}&page={1}'.format(id, page)
          super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id="", width=854, height=480, fs=1)
  print('Video available at https://www.bilibili.com/video/{0}'.format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id="hfntEL8TVvY", width=854, height=480, fs=1, rel=0)
  print('Video available at https://youtube.com/watch?v=' + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

## Think! 3: Determine your basic ingredients.

Determine the basic ingredients. This will allow you to think deeper about what your model will need. It's a crucial step before you can formulate hypotheses because you first need to understand what the model will need to contain. There are 2 aspects you want to think about:

1. What parameters / variables are needed in the model?
  * Constants?
  * Do they change over space, time, conditions…?
  * What details can be omitted?
  * Constraints, initial conditions?
  * Model inputs / outputs?
2. Variables needed to describe the process to be modelled?
  * Brainstorming!
  * What can be observed / measured?  latent variables?    
  * Where do these variables come from?
  * Do any abstract concepts need to be instantiated as variables?
    * E.g. value, utility, uncertainty, cost, salience, goals, strategy, plant, dynamics
    * Instantiate them so that they relate to potential measurements! 

This is a step where your prior knowledge and intuition is tested. You want to end up with an inventory of *specific* concepts and/or interactions that need to be instantiated. 

>### Make sure to avoid the following pitfalls
* I’m experienced, I don’t need to think about ingredients anymore
  * Or so you think…
* I can’t think of any ingredients
  * Think about the potential experiment. What are your stimuli? What parameters? What would you control? What do you measure?
* I have all inputs and outputs
  * Good! But what will link them? Thinking about that will start shaping your model and hypotheses
* I can’t think of any links (= mechanisms)
  * You will acquire a library of potential mechanisms as you keep modeling and learning
  * But the literature will often give you hints through hypotheses
  * If you still can't think of links, then maybe you're missing ingredients?

The hardest part is Step 1. Once that is properly set up, all other should be easier. **BUT**: often you think that Step 1 is done only to figure out in later steps (anywhere really) that you were not as clear on your question and goal than you thought. Revisiting Step 1 is frequent necessity. Don't feel bad about it. You can revisit Step 1 later; for now, let's move on to the nest step.

**This should take approximately 30 min.**

----
#Step 4: Formulating specific, mathematically defined hypotheses



In [ ]:
# @title Video 3: Formulating a hypothesis
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
      def __init__(self, id, page=1, width=400, height=300, **kwargs):
          self.id=id
          src = 'https://player.bilibili.com/player.html?bvid={0}&page={1}'.format(id, page)
          super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id="", width=854, height=480, fs=1)
  print('Video available at https://www.bilibili.com/video/{0}'.format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id="Vc3g1XajLlc", width=854, height=480, fs=1, rel=0)
  print('Video available at https://youtube.com/watch?v=' + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

## Think! 4: Formulating your hypothesis

Once you have your question and goal lines up, you have done a literature review (let's assume for now) and you have thought about ingredients needed for your model, you're now ready to start thinking about *specific* hypotheses.

Formulating hypotheses really consists of two consecutive steps:
1. You think about the hypotheses in words by relating ingredients identified in Step 3
  * What is the model mechanism expected to do? 
  * How are different parameters expected to influence model results?

2. You then express these hypotheses in mathematical language by giving the ingredients identified in Step 3 specific variable names. 
  * Be explicit, e.g. y(t)=f(x(t),k) but z(t) doesn’t influence y


There are also "structural hypotheses" that make assumptions on what model components you hypothesize will be crucial to capture the phenomenon at hand. 

**Important**: Formulating the hypotheses is the last step before starting to model. This step determines the model approach and ingredients. It provides a more detailed description of the question / goal from Step 1. The more precise the hypotheses, the easier the model will be to justify. 

>### To succeed here, avoid the following pitfalls
* I don’t need hypotheses, I will just play around with the model
  * Hypotheses help determine and specify goals. You can (and should) still play…
* My hypotheses don’t match my question (or vice versa)
  * This is a normal part of the process!
  * You need to loop back to Step 1 and revisit your question / phenomenon / goals
* I can’t write down a math hypothesis
  * Often that means you lack ingredients and/or clarity on the hypothesis
  * OR: you have a “structural” hypothesis, i.e. you expect a certain model component to be crucial in explaining the phenomenon / answering the question

**This step should take about 30 min**.






**We now have everything we need to actually start modelling!**



In [ ]:
# @title Video 4: Next step!
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
      def __init__(self, id, page=1, width=400, height=300, **kwargs):
          self.id=id
          src = 'https://player.bilibili.com/player.html?bvid={0}&page={1}'.format(id, page)
          super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id="", width=854, height=480, fs=1)
  print('Video available at https://www.bilibili.com/video/{0}'.format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id="bDqwjCYhyAg", width=854, height=480, fs=1, rel=0)
  print('Video available at https://youtube.com/watch?v=' + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

----
#Summary

In this tutorial, we worked through some steps of the process of modeling. 

- We defined a phenomenon and formulated a question (step 1)
- We collected information the state-of-the-art about the topic (step 2)
- We determined the basic ingredients (step 3), and used these to formulate a specific mathematically defined hypothesis (step 4)

You are now in a position that you could start modeling without getting lost. But remember: you might have to work through steps 1-4 again after doing a literature review and/or if there were other pitfalls you identified along the way (which is totally normal).

[In the next tutorial](https://colab.research.google.com/drive/10689_o9Aea0PImoVhZ26IVxfK9sRHRuM?usp=sharing) we will continue with the steps 5-10 to guide you through the implementation and completion stages of the projects. 

----
#Reading
Blohm G, Kording KP, Schrater PR (2020). _A How-to-Model Guide for Neuroscience_. eNeuro, 7(1) ENEURO.0352-19.2019. https://doi.org/10.1523/ENEURO.0352-19.2019 

Kording KP, Blohm G, Schrater P, Kay K (2020). _Appreciating the variety of goals in computational neuroscience_. Neurons, Behavior, Data Analysis, and Theory 3(6). https://nbdt.scholasticahq.com/article/16723-appreciating-the-variety-of-goals-in-computational-neuroscience

Schrater PR, Peters MK, Kording KP, Blohm G (2019). _Modeling in Neuroscience as a Decision Process_. OSF pre-print. https://osf.io/w56vt/